# 📊 Telco Churn ETL Analysis Notebook
This notebook loads cleaned churn data from Supabase, performs analysis, and saves outputs.

In [1]:

# 🔌 Connect to Supabase
from supabase import create_client
from dotenv import load_dotenv
import pandas as pd
import os

load_dotenv()

url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")

supabase = create_client(url, key)

print("✅ Connected to Supabase")


✅ Connected to Supabase


In [2]:

# 📥 Load cleaned dataset from Supabase

response = supabase.table("telco_churn_cleaned").select("*").execute()
df = pd.DataFrame(response.data)

print("Rows loaded:", len(df))
df.head()


Rows loaded: 1000


,id,tenure,monthlycharges,totalcharges,churn,internetservice,contract,paymentmethod,tenure_group,monthly_charge_segment,has_internet_service,is_multi_line_user,contract_type_code
0,1,1,29.85,29.85,No,DSL,Month-to-month,Electronic check,New,Low,1,0,0
1,2,34,56.95,1889.50,No,DSL,One year,Mailed check,Regular,Medium,1,0,1
2,3,2,53.85,108.15,Yes,DSL,Month-to-month,Mailed check,New,Medium,1,0,0
3,4,45,42.30,1840.75,No,DSL,One year,Bank transfer (automatic),Loyal,Medium,1,0,1
4,5,2,70.70,151.65,Yes,Fiber optic,Month-to-month,Electronic check,New,High,1,0,0


In [3]:

# 📊 Churn Percentage
churn_rate = (df["churn"].str.lower() == "yes").mean() * 100
churn_rate


np.float64(25.6)

In [4]:

# 💰 Average Monthly Charges per Contract Type
df.groupby("contract")["monthlycharges"].mean().round(2)


contract
Month-to-month    67.30
One year          68.18
Two year          63.27
Name: monthlycharges, dtype: float64

In [5]:

# 👥 Tenure Group Distribution
df["tenure_group"].value_counts()


tenure_group
New         321
Regular     268
Champion    209
Loyal       202
Name: count, dtype: int64

In [6]:

# 🌐 Internet Service Distribution
df["internetservice"].value_counts()


internetservice
Fiber optic    468
DSL            329
No             203
Name: count, dtype: int64

In [7]:

# 🔄 Churn vs Tenure Group Pivot Table
pivot = pd.crosstab(df["tenure_group"], df["churn"])
pivot


churn,No,Yes
tenure_group,,
Champion,196,13
Loyal,169,33
New,167,154
Regular,212,56


In [8]:

# 💾 Save analysis into processed folder

base_dir = os.path.dirname(os.getcwd())
processed_dir = os.path.join(base_dir, "data", "processed")
os.makedirs(processed_dir, exist_ok=True)

df.to_csv(os.path.join(processed_dir, "telco_churn_cleaned.csv"), index=False)
pivot.to_csv(os.path.join(processed_dir, "churn_tenure_pivot.csv"))

analysis_summary = pd.DataFrame({
    "metric": ["churn_percentage"],
    "value": [churn_rate]
})

analysis_summary.to_csv(os.path.join(processed_dir, "analysis_summary.csv"), index=False)

print("✅ Files saved in data/processed/")


✅ Files saved in data/processed/
